In [37]:
import numpy as np
from keras.utils import np_utils
import os, cv2
os.chdir("../")
os.getcwd()

'c:\\Users\\hp\\Documents\\iNeuron_intern'

In [13]:
from covid_md.constant import *
from covid_md.utils import read_yaml, create_directories

In [31]:
from pathlib import Path
from dataclasses import dataclass
@dataclass(frozen= True)
class DataTrainingConfig:
    root_dir: Path
    data_path: Path
    

In [32]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_data_training_config(self)->DataTrainingConfig:
        config= self.config.data_training
        create_directories([config.root_dir])
        data_training_config= DataTrainingConfig(
            root_dir= self.config.data_training.root_dir,
            data_path= self.config.data_training.data_path
        )
        return data_training_config

In [35]:
class DataTraining:
    def __init__(self, config: DataTrainingConfig):
        self.config= config
    def data_load(self):
        path= self.config.data_path
        print(path)

In [36]:
try:
    config= ConfigurationManager()
    data_training_config= config.get_data_training_config()
    data_training= DataTraining(config= data_training_config)
    data_training.data_load()
except Exception as e:
    raise e

artifacts/data_preprocess
